In [2]:
# Loading Libraries
import polars as pl
import polars.selectors as cs
from plotnine import *
from mizani.labels import label_number
import sys 

print(f'My System version is {sys.version};\nPolars version is {pl.__version__}.')

My System version is 3.12.4 (main, Jun  6 2024, 18:26:44) [Clang 15.0.0 (clang-1500.3.9.4)];
Polars version is 1.0.0.
My System version is 3.12.4 (main, Jun  6 2024, 18:26:44) [Clang 15.0.0 (clang-1500.3.9.4)];
Polars version is 1.0.0.


In [3]:
# Load the load
url = 'https://raw.githubusercontent.com/tongakuot/python_tutorials/main/00_data/ss_2008_census_data_raw.csv'

# Tell polars how to label missing values
census_raw = pl.read_csv(url, null_values='NA')

# Inspect the data
print(census_raw)

shape: (453, 10)
┌───────────────┬─────────────────────────────────┬───────────────────┬──────────┬───┬──────────┬───────┬─────────┬────────┐
│ Region        ┆ Region Name                     ┆ Region - RegionId ┆ Variable ┆ … ┆ Age Name ┆ Scale ┆ Units   ┆ 2008   │
│ ---           ┆ ---                             ┆ ---               ┆ ---      ┆   ┆ ---      ┆ ---   ┆ ---     ┆ ---    │
│ str           ┆ str                             ┆ str               ┆ str      ┆   ┆ str      ┆ str   ┆ str     ┆ i64    │
╞═══════════════╪═════════════════════════════════╪═══════════════════╪══════════╪═══╪══════════╪═══════╪═════════╪════════╡
│ KN.A2         ┆ Upper Nile                      ┆ SS-NU             ┆ KN.B2    ┆ … ┆ Total    ┆ units ┆ Persons ┆ 964353 │
│ KN.A2         ┆ Upper Nile                      ┆ SS-NU             ┆ KN.B2    ┆ … ┆ 0 to 4   ┆ units ┆ Persons ┆ 150872 │
│ KN.A2         ┆ Upper Nile                      ┆ SS-NU             ┆ KN.B2    ┆ … ┆ 5 to 9   ┆ units ┆ Pe

In [4]:
# Let's import our custom function censuspy as cy
import censuspy as cy
census = (
    cy.tweak_census(
    census_raw, 
    grouping_cols=['former_region', 'state', 'gender', 'category'], 
    condition=~((pl.col('gender') == 'Total') | (pl.col('category') == 'Total')) 
    )
)

# inspect the first 5 rows
print(census)

ModuleNotFoundError: No module named 'censuspy'